In [1]:
import requests
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from IPython.display import Image
import json

In [2]:
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.csv'

In [ ]:
##params = {"incident_reported": "Floating Timestamp", "End if not specified", 'api_key': api_key}

In [3]:
response = requests.get(endpoint)

In [4]:
print(response.text[:1000])

"primary_key","incident_number","report_type","report_type_description","incident_status_code","incident_status_description","investigation_status","incident_occurred","incident_reported","incident_location","latitude","longitude","zip_code","rpa","zone","location_code","location_description","offense_number","offense_nibrs","offense_description","weapon_primary","weapon_description","victim_number","domestic_related","victim_type","victim_description","victim_gender","victim_race","victim_ethnicity","victim_county_resident","mapped_location"
"2021260418_11","2021260418","W","WITNESS","O","OPEN","Open","2021-04-14T17:00:00.000","2021-04-21T16:40:00.000","HILLSIDE AVE","36.14","-86.78",,"6903","815","13","HIGHWAY, ROAD, ALLEY","1","240","VEHICLE THEFT","17","NONE","1","false","I","INDIVIDUAL (18 AND OVER)","M","W","Non-Hispanic","RESIDENT","POINT (-86.78 36.14)"
"20210519961_11","20210519961","S","SUSPECT","U","UNFOUNDED","Closed","2021-09-25T00:00:00.000","2021-10-04T22:59:00.000","102

In [6]:
from io import StringIO

In [8]:
burglaries = pd.read_csv(StringIO(response.text))

In [9]:
burglaries.head()

,primary_key,incident_number,report_type,report_type_description,incident_status_code,incident_status_description,investigation_status,incident_occurred,incident_reported,incident_location,...,weapon_description,victim_number,domestic_related,victim_type,victim_description,victim_gender,victim_race,victim_ethnicity,victim_county_resident,mapped_location
0,2021260418_11,2021260418,W,WITNESS,O,OPEN,Open,2021-04-14T17:00:00.000,2021-04-21T16:40:00.000,HILLSIDE AVE,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.78 36.14)
1,20210519961_11,20210519961,S,SUSPECT,U,UNFOUNDED,Closed,2021-09-25T00:00:00.000,2021-10-04T22:59:00.000,1022 1022,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.718 36.278)
2,20210519825_11,20210519825,D,DISPATCHED,U,UNFOUNDED,Closed,2021-10-04T17:01:00.000,2021-10-04T21:44:00.000,2958 2958,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,POINT (-86.778 36.359)
3,20210519800_11,20210519800,S,SUSPECT,O,OPEN,Open,2021-10-04T20:00:00.000,2021-10-04T21:18:00.000,SHELBY AVE,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,POINT (-86.77 36.17)
4,20210519773_11,20210519773,D,DISPATCHED,U,UNFOUNDED,Closed,2021-10-04T19:56:00.000,2021-10-04T20:42:00.000,200 200,...,NONE,1,True,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,POINT (-86.713 36.261)


In [ ]:
##Census- Need to find my file
with open('data/Census.json') as fi:
    credentials = json.load(fi)
    
api_key = credentials['api_key']

In [ ]:
##endpoint = https://www.census.gov/data/developers/data-sets/acs-5year.html

In [ ]:
## params = {"Key": "Type", "End if not specified", 'api_key': api_key}

In [ ]:
## Get request example- respone = requests.get(endpoint, params = params)

In [ ]:
## Response should be 200 you do not want anything that starts with a  400

In [ ]:
## response tab gives you your options. response.json() gives you dictionary